The essence of the functional paradigm can be summerized as:    
- Avoiding Mutation
- Avoiding Side Effects

### Starting simple
`add1 :: int -> int`    
A function "maps" a value in the "domain" to a value in the "range". There is no "computation".

In [ ]:
let add1 x = x + 1
add1 5

6

#### Important notes:    
`x` is not something that can change once we pass 5 in.    
This is not assignment - What is happening here is "binding".    
Once a domain value is "bound" is cannot be changed.     
`x` is a placeholder that can be referred to later, not changed.    
**There are no "variables", only values**

### Pure Functions
Remember one of the essentials of functial programming `avoiding side effects`.  This is so important there is a special name for functions that have this property. `Pure Functions` a function with no side effects.  This is the goal!

What a `side effect`?  aside from manipulating local memory (this gets a free pass), it is any change outside of the function.    
Writing to the console, a file, a database. Changing a variablefs value that resides outside of the scope of your function. etc...    

Of course you have to do these things at some point.  Though you should strive to keep as much code pure as you can, and push side effects to the very edges of your code.  There is also a helpul design pattern called `monads` that we will explore later.

### Simple Value and Function Values
Values are quantities of a specific type. `Binding` associates a name with a definition.    
Similarly - the name `add1` is just a "binding" to "the function that adds one to its input"    
"Every time you see the name `add1`, replace it with the function that adds one to its input".    

For example, we can "bind" `add1` to a new name.    
`plus1` and `add1` are "bound" to the same value, which is the function that adds one to its input.    
`plus1 :: int -> int`    

Functions are values too that can be passed around - This is a key part of thinking functionaly: functions are values that can be passed around to other functions.

In [ ]:
let plus1 = add1
let result1 = add1 2
let result2 = plus1 2
printfn $"add1: {result1}"
printfn $"plus1: {result2}"

let applyFn fn x = fn x
let result3 = applyFn add1 2
printfn $"applyFn add1: {result3}"

add1: 3
plus1: 3
applyFn add1: 3


#### Higher-Order functions
A function that takes a function as a paramater, or returns another function.   
Example:
`evalWith5ThenAdd2 :: (int -> int) -> int` 

In [ ]:
let evalWith5ThenAdd2 fn = (fn 5) + 2
evalWith5ThenAdd2 add1

8

In this function the domain is `(int -> int)` and the range is `int`.    
This means the input is a function that maps ints to ints, and the output is just an int.    

`timesThree :: int -> int`

In [ ]:
let timesThree x = x * 3
evalWith5ThenAdd2 timesThree

17

Now, lets return a function rather than use one as input.    
`adderGenerator :: int -> (int -> int)`

In [ ]:
let adderGenerator x = 
    let add a b = a + b
    add x

let add2 = adderGenerator 2
let add3 = adderGenerator 3

let result1 = add1 1
let result2 = add2 result1
let result3 = add3 result2
printf $"{result1} {result2} {result3}"

2 4 7

### Currying
If 'functionaly' we can only have one input, how do we pass multiple inputs?    
1. tuples
2. currying! - a series of functions that each take one paramater    

Example: `printTwoParameters :: int -> int -> ()`

In [ ]:
let printTwoParameters x = 
    let printParams y = 
        printfn $"{x} {y}"
    printParams

We now have a function that takes a single argument and returns a new function with that parameter baked in.  The new function also takes a single parameter.    

In [ ]:
let printOneThenY = printTwoParameters 1 // create a new func that prints 1 and whatever else you give it.
printOneThenY 2 // finish the function logic with the second parameter.
//or
printTwoParameters 1 2

1 2
1 2


### Partial Application
Currying leads to 'partial application' - a powerful concept.    
The idea is to take a function of N parameters and apply some of the arguments, returning a new function of the remaining arguments.    
This is similar to `adderGenerator` but more generalized.    

Example:    
`add :: int -> int -> int`    
partially apply 2 to get        
`add2 :: int -> int`    

`isLessThan :: int -> int -> bool`    
partially apply 5 to get    
`isLessThan5 :: int -> bool`

In [ ]:
let add x y = x + y
let add2 = add 2 // partial application

let result1 = add2 1

let isLessThan x y = x > y 
let isLessThan5 = isLessThan 5 // partial application

let result2 = isLessThan5 result1

printfn $"{result1} {result2}"

3 True


### Function Composition 
Taking the output of one function, and using it as the input of another function.    

Thes simplest form of this is a data proccesing pipeline that takes an integer and performs a series of calculations and returns a result.

#### Pipeline operator
Passing functions is great! but can get hard to read, consider the following:

In [ ]:
add3 (add2 (add1 1))

7

First, we addOne to 0 which gives us 1    
Then, we substractTwo to 1 which gives us -1    
Then, we addOne to -1 to get to 0    
Finally, weOne to 0 to get back to 1    

But, we had to read the whole line to know where to start, and then work backwards... bleh    
This is a common way to create data processing pipelines, there is even some helpful syntax: 

The Pipe! `|>` - it uses the output of one function as the input of the next.   

In [ ]:
add1 1
|> add2
|> add3

7

Much better!  Now it looks like what is actually does!    

#### Composition operator

Another form of this is taking two functions and returning a new function that is the result of binding one functions output to another functions input.

For example, if you have a function that maps from T1 -> T2 and another function that maps from T2 -> T3, then you can compose them giving you a new function that maps from T1 -> T3.

The composition operator `>>`

In [ ]:
let f x = x * 2
let g x = x + 1

let fg = f >> g
let result1 = fg 2

//similar without operator
let fg2 x = g (f x)
let result2 = fg2 2

printfn $"{result1} {result2}"

5 5


### Combinators
A combinator is a function whos results only depends on the functions parameters. That is to say, a combinator is a function that only combines its parameters in some way.    


**Combinator Birds** 

In [ ]:
/// Identity function, or the Idiot bird
let I x = x;

/// The Kestrel
let K x = fun y -> x

/// The Mockingbird
let M x = fun y -> x (x (y))

/// The Thrush
let T x = fun y -> y (x)

/// The Queer bird
let Q x = fun y -> fun z -> y (x (z))

/// The Starling
let S x = fun y -> fun z -> x (z, y (z))

/// The infamous Y-combinator, or Sage bird
let rec Y f = fun x -> f (Y (f), x)

A combinator is the safest form of a function. There is nothing outside of the function that can change its result.        
Using combitators from a combinator library is like using Lego from a lego set to create something new.    

//todo    
function signatures   
monads